# ETL da camada bronze para camada silver

Este notebook realiza o ETL dos dados da camada bronze para a camada silver. Ou seja: ele abre o dataset e o salva num dataframe, realiza a transformação dos dados e os carrega num arquivo `.csv` e no banco de dados.

### Estratégia de Leitura Otimizada (Chunking)

Para contornar as limitações de memória do ambiente WSL ao ler o arquivo `MICRODADOS_ENEM_2021.csv` (>1GB), adotamos a estratégia de leitura em lotes (*chunks*).
Diferente do padrão do Pandas que tenta carregar todo o arquivo na RAM, este método cria um iterador que processa o arquivo em fragmentos sequenciais, similar ao comportamento "lazy" do Apache Spark.

**Parâmetros Críticos:**
* `chunksize=100000`: Limita o consumo de RAM processando apenas 100 mil linhas por vez.
* `encoding='ISO-8859-1'`: Corrige erros de decodificação de caracteres que contem dentro do arquivo `MICRODADOS_ENEM_2021.csv` que e um "erro" bem comuns em dados  usado no Brasil.
* `sep=';'`: Define o ponto e vírgula como separador correto, evitando que o dataset seja lido em uma única coluna e assim trazendo como esperamos.

In [5]:
import pandas as pd
import numpy as np

data_layer_filepath = '../raw/'

chunk_size = 100000

chunks = pd.read_csv(
    data_layer_filepath + 'data_raw/MICRODADOS_ENEM_2021.csv', 
    low_memory=False, 
    encoding='ISO-8859-1', 
    sep=';', 
    chunksize=chunk_size
)

print("Leitura em chunks iniciada...")

contagem_total_linhas = 0

for df_chunk in chunks:
    print(df_chunk.head())
    
    contagem_total_linhas += len(df_chunk)



print(f"Processamento finalizado.")

Leitura em chunks iniciada...
   NU_INSCRICAO  NU_ANO  TP_FAIXA_ETARIA TP_SEXO  TP_ESTADO_CIVIL  TP_COR_RACA  TP_NACIONALIDADE  TP_ST_CONCLUSAO  TP_ANO_CONCLUIU  TP_ESCOLA  TP_ENSINO  IN_TREINEIRO  CO_MUNICIPIO_ESC NO_MUNICIPIO_ESC  CO_UF_ESC SG_UF_ESC  TP_DEPENDENCIA_ADM_ESC  TP_LOCALIZACAO_ESC  TP_SIT_FUNC_ESC  CO_MUNICIPIO_PROVA     NO_MUNICIPIO_PROVA  CO_UF_PROVA SG_UF_PROVA  TP_PRESENCA_CN  TP_PRESENCA_CH  TP_PRESENCA_LC  TP_PRESENCA_MT  CO_PROVA_CN  CO_PROVA_CH  CO_PROVA_LC  CO_PROVA_MT  NU_NOTA_CN  NU_NOTA_CH  NU_NOTA_LC  NU_NOTA_MT                                TX_RESPOSTAS_CN                                TX_RESPOSTAS_CH                                    TX_RESPOSTAS_LC                                TX_RESPOSTAS_MT  TP_LINGUA                                 TX_GABARITO_CN                                 TX_GABARITO_CH                                     TX_GABARITO_LC                                 TX_GABARITO_MT  TP_STATUS_REDACAO  NU_NOTA_COMP1  NU_NOTA_COMP2  NU_NOTA_C

In [9]:
import pandas as pd
import numpy as np

data_layer_filepath = '../raw/'


df = pd.read_csv(
    data_layer_filepath + 'data_raw/MICRODADOS_ENEM_2021.csv',
    low_memory=False,
    nrows=1000000,     
    encoding='ISO-8859-1',
    sep=';'
)

print("Amostra carregada com sucesso!")
df.head()

Amostra carregada com sucesso!


,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3144805,Nova Lima,31,MG,0,1,1,0,NaN,881.0,892.0,NaN,NaN,574.6,472.6,NaN,NaN,BBCCECEABCEABADDAAEECBEBADADAAABABBABBACCCBBB,99999BADDEAEBACADADDAEABCEECDDBBAEADEDEABADBBA...,NaN,1,NaN,BEECCAEABADBCAEBAAEEDDEBBBADBCBAAEEBBBADCCBBA,ABBACAAECACDBDDADEBDDCBDCEDBEDDBBBBDCEEAADABAC...,NaN,1.0,140.0,120.0,120.0,180.0,200.0,760.0,F,F,B,B,3.0,D,A,B,C,B,A,B,A,B,A,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2704302,Maceió,27,AL,1,1,1,1,912.0,882.0,891.0,901.0,505.9,551.8,498.3,461.5,DCBCCBDBCCCCCCDDCDCCCACCABCCECCCCAADCCCBBCADE,ECCDAAEBCDACBDBDECABBEAACBCBCCCDACEABEBEBACAE,99999ACECCEEBACDABACBECBBCDBDEBDDCCCDECCDDCCCC...,CDCCBCCCDCCCCBEACEECAEABECCCDCBBCABACDAEDCCEC,1,DABCEDEBEEBBCABEDDCBCBECDADCDAACBDCCCDBBBEBAB,EBAAEADBCACBBABEECBAAEEBBBADCBADBCEDDEBBCAEAB,ACABBACAEADCEADABDACDCEABDDADBEDDDEBBACCDDDCCE...,DCCAEBABDDCABEECCBCCEXADDCEECDEBADCABBDBDEDCE,1.0,120.0,120.0,120.0,120.0,80.0,560.0,B,B,B,B,3.0,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3515707,Ferraz de Vasconcelos,35,SP,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,C,C,B,3.0,C,A,B,B,A,A,B,B,A,A,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,1.0,0,2304202.0,Crato,23.0,CE,2.0,1.0,1.0,2304202,Crato,23,CE,1,1,1,1,911.0,880.0,890.0,902.0,580.7,678.9,638.9,659.5,CCABBDAEDBEBEDDCAAABBDAADBDCACACEDAABCEABAEBC,CBBDABAABBAAEECEBAADADBEBCAEEBEDDEBBBADBCBEDC,ABBCA99999CDECCCBADCDBCEAEBBAAEADEEADDACAEDDAB...,EACDCEACDDDDCBDDCABADAEBACDCDDEDBBCDBECACCCCA,0,CDBBBCABEDEDEAAEBACCCBDABCDCBCBDECDADCDBBEEBB,BBBAAEECBBABEECEBAAEADBCACAEABEDDEBBBADBCBADC,ABBCAEAACADDCACDCADABBBBDEBBACEADCEBDDACEEDDDB...,ECCCABBDBDDDCEDDCABXEEEBABEDCEDCCAEBADBCCECDA,1.0,120.0,180.0,120.0,200.0,160.0,780.0,B,B,B,B,6.0,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,1.0,0,2311603.0,Redenção,23.0,CE,2.0,1.0,1.0,2300150,Acarape,23,CE,1,1,1,1,912.0,882.0,891.0,901.0,497.7,532.4,457.6,582.6,BABCDCCBEBCD.DEDCABCBDEDAABEDADBDBCDECCAEDBCB,CAE.ACDCAAEBBEDEBCADADEDADAACEBDABBCEABBCDEAC,99999EAACBDCBECCABADBEDADCDADDCDDECABAADEBBDAC...,ACEDEBECABDABDEECBDEEADDDDBEBCEADCCADCADEDCEB,1,DABCEDEBEEBBCABEDDCBCBECDADCDAACBDCCCDBBBEBAB,EBAAEADBCACBBABEECBAAEEBBBADCBADBCEDDEBBCAEAB,ACABBACAEADCEADABDACDCEABDDADBEDDDEBBACCDDDCCE...,DCCAEBABDDCABEECCBCCEXADDCEECDEBADCABBDBDEDCE,1.0,120.0,140.0,160.0,180.0,180.0,780.0,D,E,F,D,4.0,C,A,B,C,A,A,B,A,B,A,B,A,A,B,A,B,E,A,B,B


In [10]:
df.rename(

    columns={col: col.lower().replace(' ', '_') for col in df.columns},

    inplace=True

)

print(df.columns)

Index(['nu_inscricao', 'nu_ano', 'tp_faixa_etaria', 'tp_sexo', 'tp_estado_civil', 'tp_cor_raca', 'tp_nacionalidade', 'tp_st_conclusao', 'tp_ano_concluiu', 'tp_escola', 'tp_ensino', 'in_treineiro', 'co_municipio_esc', 'no_municipio_esc', 'co_uf_esc', 'sg_uf_esc', 'tp_dependencia_adm_esc', 'tp_localizacao_esc', 'tp_sit_func_esc', 'co_municipio_prova', 'no_municipio_prova', 'co_uf_prova', 'sg_uf_prova', 'tp_presenca_cn', 'tp_presenca_ch', 'tp_presenca_lc', 'tp_presenca_mt', 'co_prova_cn', 'co_prova_ch', 'co_prova_lc', 'co_prova_mt', 'nu_nota_cn', 'nu_nota_ch', 'nu_nota_lc', 'nu_nota_mt', 'tx_respostas_cn', 'tx_respostas_ch', 'tx_respostas_lc', 'tx_respostas_mt', 'tp_lingua', 'tx_gabarito_cn', 'tx_gabarito_ch', 'tx_gabarito_lc', 'tx_gabarito_mt', 'tp_status_redacao', 'nu_nota_comp1', 'nu_nota_comp2', 'nu_nota_comp3', 'nu_nota_comp4', 'nu_nota_comp5', 'nu_nota_redacao', 'q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009', 'q010', 'q011', 'q012', 'q013', 'q014', 'q015',
  

In [11]:
df.rename(
    columns={col: col.lower() for col in df.columns}, 
    inplace=True
)

print(df.columns)

Index(['nu_inscricao', 'nu_ano', 'tp_faixa_etaria', 'tp_sexo', 'tp_estado_civil', 'tp_cor_raca', 'tp_nacionalidade', 'tp_st_conclusao', 'tp_ano_concluiu', 'tp_escola', 'tp_ensino', 'in_treineiro', 'co_municipio_esc', 'no_municipio_esc', 'co_uf_esc', 'sg_uf_esc', 'tp_dependencia_adm_esc', 'tp_localizacao_esc', 'tp_sit_func_esc', 'co_municipio_prova', 'no_municipio_prova', 'co_uf_prova', 'sg_uf_prova', 'tp_presenca_cn', 'tp_presenca_ch', 'tp_presenca_lc', 'tp_presenca_mt', 'co_prova_cn', 'co_prova_ch', 'co_prova_lc', 'co_prova_mt', 'nu_nota_cn', 'nu_nota_ch', 'nu_nota_lc', 'nu_nota_mt', 'tx_respostas_cn', 'tx_respostas_ch', 'tx_respostas_lc', 'tx_respostas_mt', 'tp_lingua', 'tx_gabarito_cn', 'tx_gabarito_ch', 'tx_gabarito_lc', 'tx_gabarito_mt', 'tp_status_redacao', 'nu_nota_comp1', 'nu_nota_comp2', 'nu_nota_comp3', 'nu_nota_comp4', 'nu_nota_comp5', 'nu_nota_redacao', 'q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009', 'q010', 'q011', 'q012', 'q013', 'q014', 'q015',
  

In [ ]:
import pandas as pd
import os
from psycopg import connect, sql
from dotenv import load_dotenv
import sys


DB_SCHEMA = "silver" 
TABLE_NAME = "dados_inep" 
TABLE_FULL_NAME = f"{DB_SCHEMA}.{TABLE_NAME}"

print("--- Iniciando processo de carga e verificação no PostgreSQL ---")


try:

    ddl_path = data_layer_filepath + 'silver/silver_ddl.sql' 
    ddl = open(ddl_path).read().replace('\n', ' ')
except Exception as e:
    print(f'Erro ao abrir arquivo ddl: {e}')
    ddl = None 

def get_db_connection_info():
    load_dotenv()
    
    url = os.getenv('DB_URL')
    db_env = os.getenv('DB_ENV')
    if url is not None and db_env == 'prod':
        return url

   
    DB_USER = "admin"           
    DB_PASSWORD = "l1l2r1r2"    
    DB_HOST = "localhost"       
    DB_PORT = "5432"
    DB_NAME = "dados_inep"

    conn_string = f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={DB_PORT}"
    return conn_string

conn_info = get_db_connection_info()


print(f"Total de linhas a serem carregadas: {len(df)}")

cols = list(df.columns)

insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
    sql.Identifier(DB_SCHEMA, TABLE_NAME),                      
    sql.SQL(", ").join(map(sql.Identifier, cols)),               
    sql.SQL(", ").join(sql.Placeholder() * len(cols))            
)

def get_number_of_rows(cur):
    
    try:
        query = sql.SQL('select count(*) from {};').format(sql.Identifier(DB_SCHEMA, TABLE_NAME))
        cur.execute(query)
        return cur.fetchone()[0]
    except:
        return 0

with connect(conn_info) as conn:
    print("\nConexão com o PostgreSQL (dados_inep) estabelecida.")
    
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE SCHEMA IF NOT EXISTS {};").format(sql.Identifier(DB_SCHEMA)))
        
        if ddl:
            print("Aplicando DDL...")
            cur.execute(ddl)
            conn.commit()
            print("Estrutura do banco de dados verificada.")
        
        print(f'Existem {get_number_of_rows(cur)} linhas na tabela {TABLE_FULL_NAME}!')
        print("Iniciando carga de dados...")

        
        counter = 0
        for _, row in df.iterrows():
            values = [None if pd.isna(v) else v for v in row]
            cur.execute(insert_query, values)
            
            counter += 1
            if counter % 10000 == 0:
                conn.commit()
                print(f"{counter} linhas processadas...")

        conn.commit()
        print("Carga concluída com sucesso!")
        print(f'Total final: {get_number_of_rows(cur)} linhas no banco de dados!')